## 중앙값으로 대체하기

In [27]:
import pandas as pd
import os
import numpy as np

out_path = r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\interpolate"

os.chdir(r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\C\\")
read_file = pd.read_excel("Face_1W_C1_S1.xlsx", index_col=0)

columns_to_interpolate = [
    'right eye.x', 'right eye.y', 'left eye.x', 'left eye.y',
    'nose.x', 'nose.y', 'mouse.x', 'mouse.y', 
    'right ear.x', 'right ear.y', 'left ear.x', 'left ear.y',
    'box_origin.x', 'box_origin.y', 'box.width', 'box.height'
]

# 각 컬럼의 결측치를 중앙값으로 채움
# for column in columns_to_interpolate:
#     median_value = read_file[column].median()
#     read_file[column].fillna(median_value, inplace=True)

#print(read_file)

os.chdir(out_path)
read_file.to_excel("Face_1W_C1_S1.xlsx")


## 얼굴 기울기의 NaN 값을 보간할 수 있는 코드

In [28]:
import os 
import pandas as pd

out_path = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/interpolate/G/"
if not os.path.exists(out_path):
    os.makedirs(out_path)
# D:\MultiModal\MultiModal_Model\results\face\face_landmark\interpolate\C

for week in ["4W"]: #"1W", "2W", "3W", "4W"
    for group in ["G4"]: # "G1","G2","G3",
        # "C1", "C2", "C3", "C4", "A1", "A2", "A3", "A4"
        for step in ["S1", "S2", "S3"]:
            os.chdir(r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\G\\")
            
            read_file = pd.read_excel(f"Face_{week}_{group}_{step}.xlsx", index_col=0)
            # NaN 값이 있는 곳들을 보간하는 부분.

            columns_to_interpolate = [
            'right eye.x', 'right eye.y', 'left eye.x', 'left eye.y',
            'nose.x', 'nose.y', 'mouse.x', 'mouse.y', 
            'right ear.x', 'right ear.y', 'left ear.x', 'left ear.y',
            'box_origin.x', 'box_origin.y', 'box.width', 'box.height'
            ]

            # quadratic
            read_file[columns_to_interpolate] = read_file[columns_to_interpolate].interpolate(method='polynomial', order=2)
            
            # 각 컬럼의 결측치를 중앙값으로 채움
            #for column in columns_to_interpolate:
                # median_value = read_file[column].mean()
                # read_file[column].fillna(median_value, inplace=True)
            
            # 보간된 값을 다시 엑셀로 내보내기. 
            os.chdir(out_path)
            read_file.to_excel(f"Face_{week}_{group}_{step}.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'Face_4W_G4_S1.xlsx'

## Face 값 중 너무 큰 값으로 튀는 outlier 제거

In [ ]:
import pandas as pd
import numpy as np
import os
#import seaborn as sns

# 데이터 표준화를 위해 scipy.stats 라이브러리 사용
import matplotlib.pyplot as plt
import scipy.stats as stats

## 이상치 제거 전, 불필요한 열을 제거하자.

In [ ]:
# file_list = ["C"] # A, D, E, F, G,
# humans = ["C1", "C2", "C3", "C4"]

#"A1", "A2", "A3", "A4"
#"B1", "B2", "B3", "B4",
#"C1", "C2", "C3", "C4",
#"D1", "D2", "D3", "D4",
#"E1", "E2", "E3", "E4",
#"F1", "F2", "F3", "F4",
#"G1", "G2", "G3", "G4"

# 불필요한 열들을 제거한 후, 최종적으로 파일들을 저장해둘 경로.
# for out in file_list:
#     for week in ["1W", "2W", "3W", "4W"]:
#         filtered_list = [human for human in humans if human.startswith(out)]
#         for human in filtered_list:
#             out_path = f"D:/MultiModal/filtered_df/{out}/" # 최종경로. 
#             # out_path = f"D:/MultiModal/filtered_df/{out}/{steps}_{human}/" # 최종경로. 

#             if not os.path.exists(out_path):
#                 os.makedirs(out_path)
                
out_path = r"D:/MultiModal/filtered_df/G/"     
                
# face 파일들이 있는 구간.
for files in ["G"]:
    default_path = f"D:/MultiModal/MultiModal_Model/results/face/face_landmark/interpolate/{files}/"
    path1 = os.listdir(default_path)
    
    # # path1 안에 들어 있는 '파일 명'만 있는 것이지 그 외는 아님. 
    for file in path1:
        if '.xlsx' in file:
            #print(file) # 파일명 출력해줌,. 
            full_path = os.path.join(default_path, file)
            #print(full_path)
            b_rotation = pd.read_excel(full_path)
            #print(b_rotation)
            filtered_df = b_rotation.drop(['right eye.x', 'right eye.y', 'left eye.x', 'left eye.y', 'nose.x', 'nose.y', 'mouse.x', 'mouse.y', 'right ear.x',
                                        'right ear.y', 'left ear.x', 'left ear.y', 'box_origin.x', 'box_origin.y'], axis = 1)
            
            filtered_df.to_excel(os.path.join(out_path, f"{file}"), index=False)

## 이상치를 제거 코드 (파일 경로 접근 + 제거)

In [ ]:
# outlier를 제거하는 부분 다시 해볼 것.

# df = 데이터 프레임 넣기, column => outlier를 제거하기 위한 열 값 할당. z_score_threshold -> 기준 값 할당.
def correct_outliers(df, column, z_score_threshold=2):
    
    mean = df[column].mean() ## 열 값에 해당하는 평균 
    std = df[column].std() ## 열 값에 해당하는 std 값 구하기 .(std* 표준편차)

    # Calculate the Z-scores
    z_scores = (df[column] - mean) / std  ## 열 값이 평균으로 부터 얼만큼 일탈되었는가 (멀어져 있는가>? )

    # z_score 값이 얼마인지 확인해볼 것. 
    print(z_scores)
    
    # Identify outliers
    # Outlier 제거하는 부분에 있어서 과하게 짤린 것 같음. =========================================================================
    outliers = abs(z_scores) > z_score_threshold ## 2보다 크면 많이 평균으로부터 많이 떨어져 있고, 매우 크거나 작다는 것을 의미. 
    #print(outliers)
    # Replace outliers with the median of the column
    median_value = df[column].median() ## 중앙값 구하기. 
    df.loc[outliers, column] = median_value ### df.loc() = median_value를 넣어줌. 

    return df

In [ ]:
import re
import math
import os
import pandas as pd

#steps = ["1W", "2W", "3W", "4W"]
#humans = ["B1", "B2", "B3", "B4"]

#   "A1", "A2", "A3", "A4",
#   "B1", "B2", "B3", "B4",
#   "C1", "C2", "C3", "C4",
#   "D1", "D2", "D3", "D4",
#   "E1", "E2", "E3", "E4",
#   "F1", "F2", "F3", "F4",
#   "G1", "G2", "G3", "G4"          


# load_path = "D:/MultiModal/filtered_df/"

# for group in ["A"]:   # "B", "C", "D", "E", "F", "G"
#     for step in steps:
#         filtered_human = [human for human in humans if human.startswith(group)]
#         for human in filtered_human:
#             dir 
#             if os.path.exists(load_path):
#                 os.makedirs(load_path)


for group in ["G"]:   # "B", "C", "D", "E", "F", "G"
    #for step in steps:
        #filtered_human = [human for human in humans if human.startswith(group)]
        #for human in filtered_human:

    load_path = f"D:/MultiModal/filtered_df/{group}/"
    #out_path = r"D:/MultiModal/filtered_df/A/" 
    
    # 작업 디렉토리 가져오기. 
    file_list = os.listdir(load_path)
    #print(file_list)
    
    # 아웃 라이어 제거한 C 그룹 경로. 
    output_folder = f"D:/MultiModal/Face_z_score/G_1/"
    # f"D:/MultiModal/Face_z_score/A_1/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in file_list:
        dir = os.path.join(load_path, file)
        if dir.split('.')[-1] == "xlsx": 
            face_df = pd.read_excel(dir)

            # 뽑아내고 싶은 열 이름 모음. 
            coordinate_columns = ["box.width", "box.height"]

            for column in coordinate_columns:
                # 뽑아낼 열 이름 넣어주고, 저장. 
                filtered_df = correct_outliers(face_df, column)
                
                #print(filtered_df)
                # filtered_df 안에 값을 넣어서, 저장해줌. 
            filtered_df.iloc[:, 1:].to_excel(output_folder + file.split('.')[0] + '.xlsx')

## 아래 코드는 내가 직접 짜본 '데이터 정규화' 코드 || 문제점 : 정규화를 하는 것은 좋은데 값이 일부 날려보냄

In [ ]:
groups = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
steps = ['1W', '2W', '3W', '4W']
humans = ['A1', 'A2', 'A3', 'A4',
          'B1', 'B2', 'B3', 'B4',
          'C1', 'C2', 'C3', 'C4',
          'D1', 'D2', 'D3', 'D4',
          'E1', 'E2', 'E3', 'E4',
          'F1', 'F2', 'F3', 'F4',
          'G1', 'G2', 'G3', 'G4']

# 저장할 수 있는 경로 설정.
for group in groups:
    for step in steps:
        for human in humans:
            result_out_path = f"D:/MultiModal/Face_z_score/{group}/{step}_{human}/"
            if not os.path.exists(result_out_path):
                os.makedirs(result_out_path)

# 경로 설정 
for group in groups:
    for step in steps:
        for human in humans:
            face_file_path = f"D:/MultiModal/filtered_df/{group}/{step}_{human}/"
            
            if os.path.exists(face_file_path):
                files = os.listdir(face_file_path)
                # 엑셀 파일들 리스트 출력하기
                #print(files)     
                
                # files -> listdir()를 통해서 list 형태로 있는 값들을 불러온 것. 
                for i in files:
                    full_path1 = os.path.join(face_file_path, i) # 경로에 있는 값 하나씩 불러오기. 
                    temp_file = pd.read_excel(full_path1) # 하나씩 불러왔으니, 이를 읽기 위해 read_excel 사용. 
                    #print(temp_file)
            
                    # 이상치를 구하기 위한 식) : (값 - 평균) / (표준편차)  => z_score 값
                    outlier_face = temp_file[(abs((temp_file['box.height'] - np.mean(temp_file['box.height'])) / np.std(temp_file['box.height']))) > 1.96].index
                    # 이상치로 판단되는 값을 제거. 
                    new_df = temp_file.drop(outlier_face)
                    #print(new_df)
                    
                    #이상치를 제거했다면, 결과 값을 엑셀에 저장. 
                    new_df.to_excel(os.path.join(result_out_path, f"filtered_outlier_{i}"), index= False)

## 델타 값 계산해서, 그래프 그리기.

In [ ]:
import seaborn as sns

# body_lean_delta 값 계산
body_path = "D:\\MultiModal\\body_lean_all\\F\\"
# B,C,D,E,F,G

# face_lean_delta 값 계산
face_path = "D:/MultiModal/filtered_df/F/"
# B,C,D,E,F,G

out_path = "D:/MultiModal/data_visualize/F/"
if not os.path.exists(out_path):
    os.makedirs(out_path) 

# 각각 뽑아봐야 하는 것들. 
steps = ['4W'] # '2W', '3W', '4W'
humans = ['F1']
# 'A2', 'A3', 'A4'

for step in steps:
    for human in humans:
        dir = os.path.join(face_path, f"{step}_{human}")
        file_list = os.listdir(dir)
        
        dir_body = os.path.join(body_path, f"{step}_{human}")
        file_body_list = os.listdir(dir_body)
        
        for file_b, files in zip(file_body_list, file_list):
            file_zip_b = os.path.join(dir_body, file_b)
            file_dir_b = pd.read_excel(file_zip_b)
            
            # body에서의 각 델타 값을 계산. 
            filtered_delta_b = file_dir_b.diff()
            
            # 그래프를 그리긱 전 NaN 값을 제거. *보간. 
            filtered_delta_b.interpolate(inplace=True)

            # 차이 값이 계산되는지 확인. 
            #print(filtered_delta_b)
            
            file_zip = os.path.join(dir, files)
            read_dir = pd.read_excel(file_zip)
            
            # 불필요한 열 제거. 
            read_dir.drop(['Unnamed: 0'], axis=1, inplace= True)
            
            # 각 델타 값들을 계산
            filtered_delta = read_dir.diff()
            print(filtered_delta) # 비교할 값이 없다면, NaN 값으로 출력. 

            # 그래프를 그리긱 전 NaN 값을 제거. 
            filtered_delta.interpolate(inplace=True)
        
            # 두 개의 데이터 프레임의 최소값을 맞춰주는 것.     
            min_length = min(len(filtered_delta), len(filtered_delta_b))
            filtered_delta = filtered_delta[:min_length]
            filtered_delta_b = filtered_delta_b[:min_length]
    
            # 산점도를 확인해볼 수 있는 것. 
            #lt.scatter(filtered_delta, filtered_delta_b)
            #plt.xlabel('body_lean')  # 바디 기울기.
            #plt.ylabel('face_lean')  # 얼굴 기울기. 
            #plt.title('Scatter Plot of face_diff() vs. body_lean_diff()')
            #plt.grid(True)  # Enable grid
            #plt.show()
        
            new_df =  pd.concat([filtered_delta, filtered_delta_b], axis=1)
            new_df.columns = ['filtered_delta', 'filtered_delta_b']
            
            # 뽑아보고 싶은 특정 열 선택. 
            data_range = new_df.loc[900:1000]
            # Calculate the correlation matrix
            correlation_matrix = new_df.corr()
            plt.figure(figsize=(10, 6))
            
            plt.plot(data_range['filtered_delta'], label='filtered_delta')
            plt.plot(data_range['filtered_delta_b'], label='filtered_delta_b')
            plt.xlabel('Index')
            plt.ylabel('Values')
            plt.title('Graph of face box and body_lean (Index Range: 5100-5450)')
            plt.legend()
            plt.grid(True)
            plt.show()
                    
        
        
            # 히트 맵으로 보여주는 코드 부분
            # 양의 상관 관계는 따뜻한 색상, 음의 상관관계는 파란색으로 나타남.
            # 값이 1또는 -1에 가까울 수록 강한 상관관계를 나타내고, 0에 가까울수록 약한 상관 관계를 나타냄. 
            # Plotting heatmap of correlation matrix
            # plt.figure(figsize=(8, 6))
            # sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
            # plt.title('Correlation Heatmap between filtered_delta and filtered_delta_b')
            # plt.show()
    

## face box scaling 해주는 구간.

In [ ]:
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

for group in ["A"]: #A
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 
    
    ## D:\MultiModal\MultiModal_postprocessing\results\face\face_landmark\G
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())

    # 출력할 파일 불러오기
    output_folder = f"D:/MultiModal/Face_z_score/Face_lean/{group}/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 컬럼명
    face_landmark_col = ["Week", "ID", "Step", "Face_lean_width", "Face_lean_height"]

    data = []
    for file in file_list:
        if file.split('.')[-1] == "xlsx":
            landmark_df = pd.read_excel(file)
            # scaling
            landmark_df["lean_width"] = min_max_scaling(landmark_df["box.width"]) # 너비
            landmark_df["lean_height"] = min_max_scaling(landmark_df["box.height"]) # 높이
            
            file_name = file.split('.')[0].split('_')
            ## + '_' + file_name[-1]
            data.append([file_name[1], file_name[2], file_name[3] if (file_name[3] == "Baseline" or file_name[3] == "S3") else file_name[3], np.mean(landmark_df["lean_width"]), np.mean(landmark_df["lean_height"])])

    avg_box_df = pd.DataFrame(data, columns=face_landmark_col)
    avg_box_df.to_excel(output_folder + f"{group}_average_face_lean.xlsx")

## 평균 Face lean 값을 정리하여 뽑아보는 구간

In [ ]:
# all df concat

group_df = []

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    os.chdir(f"D:/MultiModal/Face_z_score/Face_lean/{group}/") 
    ## D:\MultiModal\MultiModal_postprocessing\results\face_time\face_landmark\G
    group_df.append(pd.read_excel(f"{group}_average_face_lean.xlsx").iloc[:, 1:])

    output_folder = f"../"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

all_group_df = pd.concat(group_df)
all_group_df.to_excel(output_folder + "All_average_face_lean(All group, anaysis).xlsx")

## 데이터 분석용으로 출력하는 파일

In [ ]:
os.chdir("D:/MultiModal/Face_z_score/Face_lean/") 
all_df = pd.read_excel("All_average_face_lean(All group, anaysis).xlsx") 

step_list = ["S1", "S2" "S3"]

df_index = ["Group", "ID"] + [f"{week}_{step}_Face_lean_{option}" for week in ["1W", "2W", "3W", "4W"] for step in step_list for option in ["width", "height"]] 

data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    for id in [f"{group}{id_index}" for id_index in range(1, 5)]:
        week_data = []
        for week in [f"{w_index}W" for w_index in range(1, 5)]:
            filtered_df = all_df[(all_df["Week"] == week) & (all_df["ID"] == id) & (all_df["Group"] == group)]
            print(filtered_df)
            week_data.extend(filtered_df[[f"{step}_Face_lean_{option}" for step in step_list for option in ["width", "height"]]].values.flatten().tolist())
        
        data.append([group, id] + week_data)

# all_summarize_df = pd.DataFrame(data, columns=df_index)
# all_summarize_df.to_excel("All_average_face_lean(All group, excel_anaysis).xlsx")

## 얼굴 기울기 횟수 세는 구간.

In [ ]:
# 얼굴 크기 변화량 값 계산하는 구간. 
def plus_minus(df):
    df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )

    W = "box.width"
    H = "box.height"

    df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
    df_new = df.iloc[1:, :]["box delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 
    
    for i in range(len(df_new)):
 
        if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -600):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        else:
            df_new.iloc[i] = 0 # 아무것도 아닐 때
        
    return df_new

### 연습해본 코드 

In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

output_folder = f"D:/MultiModal/Face_z_score/Face_lean_count/C/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["C"]: # , "B","C", "D", "E", "F", "G"
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 

    # 스텝에 나와 있는 것 
    step_list = ["S1", "S2", "S3"]
    #step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

    face_index = ["Group", "Week",] + [f"{step}_Face_lean_4p" for step in step_list] + [f"{step}_Face_lean_3p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        
        step_data_4p = []
        step_data = []
        for step in step_list:
            df1 = pd.read_excel(f'Face_{week}_{group}1_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'Face_{week}_{group}2_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'Face_{week}_{group}3_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'Face_{week}_{group}4_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}4_{step}.xlsx') else []
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
            #length = a.min()
            length = a.min()-1 # 최솟값에서 -1을 해줌. 

            # def문을 활용해서 delta 값 만들기. 
            def plus_minus(df):
                df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )
 
                W = "box.width"
                H = "box.height"

                df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
                df_new = df.iloc[1:, 18] # ? 왜 열 값을 이렇게 주었을까? 첫 1번 열과 18개의 행만  
                
                # for i in range(len(df_new)):
                #     if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
                #         df_new.iloc[i] = 1 # 앞으로 기울었을 때 
                #     elif(df_new.iloc[i] < -600):
                #         df_new.iloc[i] = -1 # 뒤로 기울었을 때 
                #     else:
                #         df_new.iloc[i] = 0 # 아무것도 아닐 때
                # return df_new 
            
            if len(df1) > 0:
                df1 = plus_minus(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"box delta_C1": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"box delta_C2": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"box delta_C3": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"box delta_C4": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
            
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = 1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            #print(df_concat.head())

            df_concat['same'] = 0 
                        
            for i in range (len(df_concat)):
                
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                df_concat.iloc[i,4] = count.max()
            
            same = np.array(df_concat["same"])
            #step_data.append((df_concat['same'] >= 3).sum())
            
            # 4명 모두 기울인 것.
            step_data_4p.append(np.where(same>=4,1,0).sum())
        
        data.append([group, week] + step_data_4p)
        # 새로운 데이터 프레임을 하나 만들어줄 것. 
        count_df = pd.DataFrame(data, columns=face_index)
    
         # out_folder 위치에 face_lean이 얼마나 기울어졌는지 기록하는 구간.  
        count_df.to_excel(output_folder + "Face_lean_count.xlsx")

In [ ]:
# 아래의 경로로 변경
os.chdir("D:/MultiModal/Face_z_score/Face_lean_count/")
all_df = pd.read_excel("C_Face_lean_count.xlsx") 

#step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

step_list = ["S1", "S2", "S3"]

df_index = ["Group"] + [f"{week}_{step}_Face_lean_3p" for week in ["1W", "2W", "3W", "4W"] for step in step_list] 
### , "2W", "3W", "4W"
data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    week_data = []
    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        filtered_df = all_df[(all_df["Week"] == week) & (all_df["Group"] == group)]
        week_data.extend(filtered_df[[f"{step}_Face_lean_3p" for step in step_list]].values.flatten().tolist())
        print(filtered_df)
    
    data.append([group] + week_data)   

all_summarize_df = pd.DataFrame(data, columns=df_index)
all_summarize_df.to_excel("All_Face_lean_count(All group, excel_anaysis).xlsx")

## 몸의 기울기 값 z_score 이상치 제거 코드.

In [ ]:
def correct_outliers(df, column, z_score_threshold=3):
    
    mean = df[column].mean()
    std = df[column].std()

    z_scores = (df[column] - mean) / std
    
    outliers = abs(z_scores) > z_score_threshold

    median_value = df[column].median()
    df.loc[outliers, column] = median_value

    return df

In [ ]:
"""
Body Scaling (이상치 처리) 정리 || 값을 조정해주는 코드
"""
import re
import math

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    os.chdir(f"Users/nabi/Desktop/filtered_df/{group}/")
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())
    # 출력할 파일 불러오기
    output_folder = f"Users/nabi/Desktop/filtered_df_result/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in file_list:
        if file.split('.')[-1] == "xlsx": # xlsx 파일만 읽기
            body_df = pd.read_excel(file)

            coordinate_columns = [col for col in body_df.columns if col.endswith(('.x', '.y', '.z'))]

            for column in coordinate_columns:
                filterd_df = correct_outliers(body_df, column)

            filterd_df.iloc[:, 1:].to_excel(output_folder + file)

## 몸의 기울기 값도 똑같이 얼굴 기울기를 셀 수 있도록 해보자. 

In [ ]:
def plus_minus_1(df):
 
    # body delta라는 이름의 데이터 프레임에 lambda로 조건식 주는 곳
    #df["body delta"] = df["body_lean"].apply(lambda x: 1 if x < 90 else (-1 if x > 90 else 0))
    
    df["body delta"] = df["Body_lean"].diff()
    #print(df["body delta"])
    
    # 첫 번째 열은 비교할 수 있는 값이 없으므로, NaN값이 생길 수 있음. 이를 방지하기 위해 0 값을 할당. 
    df.loc[0, 'Body_lean'] = 0
    df_new = df.iloc[1:, :]["body delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 

    # 절대값을 계산해서 비교.
    #df_diff = abs(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 0.02):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -0.02):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        elif(-0.02 < df_new.iloc[i] < 0.02):
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    
    return df_new 

In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

# 출력할 수 있는 경로 설정 "D:/MultiModal/body_lean_all/body_lean_count/"
output_folder = f"/Users/nabi/Desktop/body_lean_count/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["A","B","C","D","E","F","G"]: # f"D:/MultiModal/body_lean_all/{group}/"
    os.chdir(f"/Users/nabi/Desktop/filtered_mean_value/{group}/")
            
    #스텝에 나와 있는 것 
    step_list = ["S1", "S2", "S3"]
    body_index = ["Group", "Week",] + [f"{step}_Body_lean_3p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        step_data = []
        for step in step_list: # lean_
            df1 = pd.read_excel(f'Mean_filtered_{week}_{group}1_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'Mean_filtered_{week}_{group}2_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'Mean_filtered_{week}_{group}3_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'Mean_filtered_{week}_{group}4_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}4_{step}.xlsx') else []
            
            #print(df1)
            #print(df2)
            #print(df3)
            #print(df4)
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
            
            length = a.min()-1 # 최솟값에서 -1을 해줌. 

            if len(df1) > 0:
                df1 = plus_minus_1(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus_1(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus_1(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus_1(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = å1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            #print(df_concat.head())

            # same 열 초기화 
            df_concat['same'] = 0 
                        
            for i in range (len(df_concat)):
                #print(i)
                # df_concat.iloc[] 번째의 값을 카운트
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                df_concat.iloc[i,4] = count.max()
            
            same = np.array(df_concat["same"])  
            # step_data 리스트 안에는 same 값이 3보다 크다면 인덱스 값을 반환해서, 만족하면 1 아니면 0을 반환. 
            step_data.append(np.where(same>=3,1,0).sum())
    
        data.append([group, week] + step_data)
        # # 새로운 데이터 프레임을 하나 만들어줄 것. 
        count_df = pd.DataFrame(data, columns=body_index)
    
        # out_folder 위치에 face_lean이 얼마나 기울어졌는지 기록하는 구간.  
        count_df.to_excel(output_folder + "All_Body_lean_count.xlsx")

In [ ]:
# 아래의 경로로 변경
os.chdir("/Users/nabi/Desktop/body_lean_count/") # "D:/MultiModal/body_lean_all/body_lean_count/"
all_df = pd.read_excel("All_Body_lean_count.xlsx") 

#step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

step_list = ["S1", "S2", "S3"]

df_index = ["Group"] + [f"{week}_{step}_Body_lean_3p" for week in ["1W", "2W", "3W", "4W"] for step in step_list] 
### , "2W", "3W", "4W"
data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    week_data = []
    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        filtered_df = all_df[(all_df["Week"] == week) & (all_df["Group"] == group)]
        week_data.extend(filtered_df[[f"{step}_Body_lean_3p" for step in step_list]].values.flatten().tolist())
    
    data.append([group] + week_data)   

all_summarize_df = pd.DataFrame(data, columns=df_index)
all_summarize_df.to_excel("All_Body_lean_count(All group, excel_anaysis).xlsx")

## body_lean의 diff() 값을 계산해보자. 

In [ ]:
import numpy as np

def show_the_diff(df):
    # body_diff 라는 이름의 열 하나 생성. || 그리고 body_lean에 대한 diff() * 열 간의 차이 값 계산. 
    df['body diff'] = df['Body_lean'].diff()
    
    # 첫 번째 열은 비교할 수 있는 값이 없으므로, NaN값이 생길 수 있음. 이를 방지하기 위해 0 값을 할당. 
    df.loc[0, 'Body_lean'] = 0
        
    # diff_df 라는 변수 안에 body diff 전체 값을 넣어 줌. 
    diff_df = df.iloc[1:, :]["body diff"]
    
    result_df = np.median(diff_df)
    result = abs(result_df)

    # 마지막 result에는 diff() 값을 반환. 
    return result

In [ ]:
import os
import pandas as pd
# 최종적으로 저장할 경로. 
data = []

for group in ["B"]: 
    os.chdir(f"/Users/nabi/Desktop/filtered_mean_value/{group}/")

    read_dir = pd.read_excel("Mean_filtered_4W_B1_S1.xlsx")
    result_diff = show_the_diff(read_dir)
    
    print(result_diff)

## 인덱스 정보 값을 넣어주기.

In [ ]:
import os
import pandas as pd

body_path = r"D:\\MultiModal\\body_lean_all\\G\\"
 
outpath = r"D:\\MultiModal\\body_lean_all\\body_lean_count\\lean(4p_2)\\G\\"
 
 
# 파일 리스트 값을 불러오기. 
list = os.listdir(body_path)

# 리스트 내에 있는 파일 하나씩 읽어오기. 
for file in list:
    read_file = os.path.join(body_path, file)
    read_f = pd.read_excel(read_file)
    
    index_val = []
    
    for i in read_f.index:
        index_val.append(i)
        
    new_df = pd.DataFrame(columns= ["Frame"])
    new_df["Frame"] = index_val
    
    result = pd.concat([new_df, read_f], axis= 1, ignore_index=False)
    result.style.hide(axis='index')
    
    result.to_excel(outpath + f"Add_{file}", index=False)
    #print(result)

## 4명 중에 N명 기울어진 정도 + Frame 시간에 맞게 뽑아낸 것


In [1]:
# 얼굴 크기 변화량 값 계산하는 구간. 
def plus_minus(df):
    df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )

    W = "box.width"
    H = "box.height"

    df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
    df_new = df.iloc[1:, :]["box delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 
    # df_new 값일 어떻게 생성되는지 확인해보기 위해 출력. 
    #print(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -600):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        else:
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    return df_new 

### Lean_time을 뽑아보는 방법 : 전체 프레임의 얼굴 크기의 평균 값 계산, 그리고 이의 5% 차이가 있을 시 lean이 발생했다고 정의

In [ ]:
import pandas as pd
import numpy as np
import os

input_folder = f"D:/MultiModal/Face_z_score/A_1/"

#path = r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\C\\"
os.chdir(input_folder)

# box size 계산하는 부분.
# df = pd.read_excel("Face_2W_C1_S1.xlsx",sheet_name="Sheet1",index_col=0)
# df["box.size"] = df["box.height"] * df["box.width"]

# df["box.size.adj"] = df['box.size'].rolling(5,center=False).mean()
# df = df.iloc[5::5]

# threshold = df["box.size.adj"].mean() * 0.05
# df_delta = df['box.size.adj'].diff().iloc[1:]
# df_lean = np.where(df_delta.abs() > threshold, df_delta/df_delta.abs() ,0)

for week in ["1W", "2W", "3W", "4W"]:
    for group in ["A1", "A2", "A3", "A4"]:
        for step in ["S1", "S2", "S3"]:
            try:
                df = pd.read_excel(f"Face_{week}_{group}_{step}.xlsx", sheet_name="Sheet1",index_col=0)
                if df.empty:
                    continue
                        
                #df = pd.read_excel(f"Face_{week}_{group}_{step}.xlsx", sheet_name="Sheet1",index_col=0)

                # box size 계산하는 부분.
                df["box.size"] = df["box.height"] * df["box.width"]

                # box size 조정하는 부분
                df["box.size.adj"] = df['box.size'].rolling(5,center=False).mean()
                df = df.iloc[5::5]

                # 평균 box size에서 5% 이상 변할 때를 확인하기 위한, threshold 값 
                threshold = df["box.size.adj"].mean() * 0.05
                
                # 이전 프레임과 다음 프레임 간의 delta 값 계산. 
                df_delta = df['box.size.adj'].diff().iloc[1:]
                
                # 그리고 threshold 보다 클 경우, delta 값 기록.
                df_lean = np.where(df_delta.abs() > threshold, df_delta/df_delta.abs() ,0)
                #print(df_lean)
                
                df_lean2 = df_lean.copy()

                def count_continuous_value(df,frame,d_range, d_threshold):
                    if np.count_nonzero(df[frame:frame+d_range] ==-1) >= d_threshold:
                        df[frame:frame+d_range] = -1
                        next_frame = frame+d_range
                        print(f"frame {(frame+1)*5} ~ frame {(frame+1)*5+d_range*5} : -1")
                    elif np.count_nonzero(df[frame:frame+d_range] ==1) >= d_threshold:
                        df[frame:frame+d_range] = 1
                        next_frame = frame+d_range
                        print(f"frame {(frame+1)*5} ~ frame {(frame+1)*5+d_range*5} : 1")
                    else:
                        df[frame] = 0
                        next_frame = frame+1
                    return next_frame

                frame = 0

                while (frame<len(df_lean2)):
                    frame = count_continuous_value(df_lean2,frame,5,3)
                

                df_lean2_df = pd.DataFrame(df_lean2, columns=['lean time'])

                #seconds = df_lean2_df.index / 25
                total_seconds = np.arange(len(df_lean2_df)) * 0.2
                
                minutes, seconds = divmod(total_seconds, 60) # divmod() 몫과 나머지를 함께 구하는 함수.
                #print(minutes)
                hours, minutes = divmod(minutes, 60)
                #print(minutes)

                # 시간 형태로 변환하여 'hh:mm:ss' 형태로 표현
                time_format = [f'{int(hours):02}:{int(minutes):02}:{seconds:04.1f}' for hours, minutes, seconds in zip(hours, minutes, seconds)]

                #print(time_format)

                df_lean2_df.index = time_format

                output_path = f"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\A_lean_time\\Face_{week}_{group}_{step}_lean.xlsx"
                df_lean2_df.to_excel(output_path, index=True)

In [ ]:
# import pandas as pd
# import numpy as np
# import os

# # Set input folder
# input_folder = "D:/MultiModal/Face_z_score/G_1/"
# output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/G_lean_time/"

# # Ensure output directory exists
# os.makedirs(output_folder, exist_ok=True)

# # Change working directory to input folder
# os.chdir(input_folder)

# def process_file(file_path, output_path):
#     try:
#         df = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
#         if df.empty:
#             return
        
#         # Calculate box size
#         df["box.size"] = df["box.height"] * df["box.width"]

#         # Adjust box size
#         df["box.size.adj"] = df['box.size'].rolling(5, center=False).mean()
#         df = df.iloc[5::5]

#         # Calculate threshold
#         threshold = df["box.size.adj"].mean() * 0.05

#         # Calculate delta
#         df_delta = df['box.size.adj'].diff().iloc[1:]

#         # Determine lean times
#         df_lean = np.where(df_delta.abs() > threshold, df_delta / df_delta.abs(), 0)
        
#         def count_continuous_value(df, frame, d_range, d_threshold):
#             if np.count_nonzero(df[frame:frame + d_range] == -1) >= d_threshold:
#                 df[frame:frame + d_range] = -1
#                 next_frame = frame + d_range
#                 print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : -1")
#             elif np.count_nonzero(df[frame:frame + d_range] == 1) >= d_threshold:
#                 df[frame:frame + d_range] = 1
#                 next_frame = frame + d_range
#                 print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : 1")
#             else:
#                 df[frame] = 0
#                 next_frame = frame + 1
#             return next_frame 

#         frame = 0
#         while frame < len(df_lean):
#             frame = count_continuous_value(df_lean, frame, 5, 3)

#         df_lean2_df = pd.DataFrame(df_lean, columns=['lean time'])

#         total_seconds = np.arange(len(df_lean2_df)) * 0.2
#         minutes, seconds = divmod(total_seconds, 60)
#         hours, minutes = divmod(minutes, 60)

#         time_format = [f'{int(h):02}:{int(m):02}:{s:04.1f}' for h, m, s in zip(hours, minutes, seconds)]
#         df_lean2_df.index = time_format

#         df_lean2_df.to_excel(output_path, index=True)

#     except Exception as e:
#         print(f"Error processing {file_path}: {e}")

# for week in ["1W", "2W", "3W", "4W"]:
#     for group in ["G1", "G2", "G3", "G4"]:
#         for step in ["S1", "S2", "S3"]:
#             file_name = f"Face_{week}_{group}_{step}.xlsx"
#             file_path = os.path.join(input_folder, file_name)
#             output_file_name = f"Face_{week}_{group}_{step}_lean.xlsx"
#             output_path = os.path.join(output_folder, output_file_name)

#             process_file(file_path, output_path)

### 전체 얼굴 크기 값에서 5% 차이가 날때 계산

In [ ]:
# import pandas as pd
# import numpy as np
# import os

# # Set input folder
# input_folder = "D:/MultiModal/Face_z_score/C_1/"
# output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/C_lean_time/"

# # Ensure output directory exists
# os.makedirs(output_folder, exist_ok=True)

# # Change working directory to input folder
# os.chdir(input_folder)

# def process_file(file_path):
#     # 데이터 프레임이 비어있을 경우, 다음과 같이 예외 처리.
#     try:
#         df = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
#         if df.empty:
#             return pd.DataFrame({"box.height": [0 for _ in range(100000)], "box.width": [0 for _ in range(100000)]})
        
#         # Calculate box size
#         df["box.size"] = df["box.height"] * df["box.width"]

#         # 5개의 열마다의 평균 값을 계산하고 있는 것.
#         df["box.size.adj"] = df['box.size'].rolling(5, center=False).mean()
#         df = df.iloc[5::5]

#         # threshold의 경우, 
#         threshold = df["box.size.adj"].mean() * 0.05

#         # Calculate delta
#         df_delta = df['box.size.adj'].diff().iloc[1:]

#         # Determine lean times
#         df_lean = np.where(df_delta.abs() > threshold, df_delta / df_delta.abs(), 0)
        
#         def count_continuous_value(df, frame, d_range, d_threshold):
#             if np.count_nonzero(df[frame:frame + d_range] == -1) >= d_threshold:
#                 df[frame:frame + d_range] = -1
#                 next_frame = frame + d_range
#                 print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : -1")
#             elif np.count_nonzero(df[frame:frame + d_range] == 1) >= d_threshold:
#                 df[frame:frame + d_range] = 1
#                 next_frame = frame + d_range
#                 print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : 1")
#             else:
#                 df[frame] = 0
#                 next_frame = frame + 1
#             return next_frame 
        
#         # 첫 프레임부터 쭉 보고자, 0 으로 설정 
#         frame = 0
#         while frame < len(df_lean):
#             frame = count_continuous_value(df_lean, frame, 5, 3)

#         df_lean2_df = pd.DataFrame(df_lean, columns=['lean time'])

#         total_seconds = np.arange(len(df_lean2_df)) * 0.2
#         minutes, seconds = divmod(total_seconds, 60)
#         hours, minutes = divmod(minutes, 60)

#         time_format = [f'{int(h):02}:{int(m):02}:{s:04.1f}' for h, m, s in zip(hours, minutes, seconds)]
#         df_lean2_df.index = time_format

#         return df_lean2_df

#     except Exception as e:
#         print(f"Error processing {file_path}: {e}")
#         return pd.DataFrame({"box.height": [0 for _ in range(100000)], "box.width": [0 for _ in range(100000)]})

# for week in ["1W","2W","3W","4W"]:
#     for group in ["C1","C2","C3","C4"]:
#         for step in ["S1", "S2", "S3"]:
#             file_name = f"Face_{week}_{group}_{step}.xlsx"
#             file_path = os.path.join(input_folder, file_name)
#             output_file_name = f"Face_{week}_{group}_{step}_lean.xlsx"
#             output_path = os.path.join(output_folder, output_file_name)

#             if os.path.exists(file_path):
#                 df = process_file(file_path)
#                 if df.empty:
#                     df = pd.DataFrame({"box.height": [0 for _ in range(100000)], "box.width": [0 for _ in range(100000)]})
#             else:
#                 df = pd.DataFrame({"box.height": [0 for _ in range(100000)], "box.width": [0 for _ in range(100000)]})

#             df.to_excel(output_path, index=True)

### 5개 중에 3개 변화

In [ ]:
import pandas as pd
import numpy as np
import os

# Set input folder
input_folder = "D:/MultiModal/Face_z_score/C_1/"
output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/cc_lean_time/"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Change working directory to input folder
os.chdir(input_folder)

def process_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
        if df.empty:
            return pd.DataFrame({"lean time": []})
        
        # Calculate box size
        df["box.size"] = df["box.height"] * df["box.width"]

        # Adjust box size
        df["box.size.adj"] = df['box.size'].rolling(5, center=False).mean()
        df = df.iloc[5::5]

        # Calculate delta
        df_delta = df['box.size.adj'].diff().iloc[1:]

        # Determine lean times based on the previous frame's box size adjusted mean
        df_lean = np.zeros(len(df_delta))
        for i in range(1, len(df_delta)):
            previous_mean = df["box.size.adj"].iloc[i-1]
            current_mean = df["box.size.adj"].iloc[i]
            threshold = previous_mean * 0.05
            delta = current_mean - previous_mean

            if abs(delta) > threshold:
                df_lean[i] = np.sign(delta)
            else:
                df_lean[i] = 0
        
        def count_continuous_value(df, frame, d_range, d_threshold):
            if np.count_nonzero(df[frame:frame + d_range] == -1) >= d_threshold:
                df[frame:frame + d_range] = -1
                next_frame = frame + d_range
                print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : -1")
            elif np.count_nonzero(df[frame:frame + d_range] == 1) >= d_threshold:
                df[frame:frame + d_range] = 1
                next_frame = frame + d_range
                print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : 1")
            else:
                df[frame] = 0
                next_frame = frame + 1
            return next_frame 

        frame = 0
        while frame < len(df_lean):
            frame = count_continuous_value(df_lean, frame, 5, 3)

        # Mark transitions from 1 to -1 with 1 and others as 0
        df_transition = np.zeros(len(df_lean))
        for i in range(1, len(df_lean)):
            if df_lean[i-1] == 1 and df_lean[i] == -1:
                df_transition[i-1] = 1
        
        df_lean2_df = pd.DataFrame(df_transition, columns=['lean time'])

        total_seconds = np.arange(len(df_lean2_df)) * 0.2
        minutes, seconds = divmod(total_seconds, 60)
        hours, minutes = divmod(minutes, 60)

        time_format = [f'{int(h):02}:{int(m):02}:{s:04.1f}' for h, m, s in zip(hours, minutes, seconds)]
        df_lean2_df.index = time_format

        return df_lean2_df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame({"lean time": []})

for week in ["1W","2W","3W","4W"]:
    for group in ["C1","C2","C3","C4"]:
        for step in ["S1", "S2", "S3"]:
            file_name = f"Face_{week}_{group}_{step}.xlsx"
            file_path = os.path.join(input_folder, file_name)
            output_file_name = f"Face_{week}_{group}_{step}_lean.xlsx"
            output_path = os.path.join(output_folder, output_file_name)

            if os.path.exists(file_path):
                df = process_file(file_path)
                if df.empty:
                    df = pd.DataFrame({"lean time": []})
            else:
                df = pd.DataFrame({"lean time": []})

            df.to_excel(output_path, index=True)

frame 315 ~ frame 340 : -1
frame 340 ~ frame 365 : 1
frame 365 ~ frame 390 : -1
frame 395 ~ frame 420 : 1
frame 690 ~ frame 715 : -1
frame 730 ~ frame 755 : -1
frame 810 ~ frame 835 : 1
frame 850 ~ frame 875 : -1
frame 875 ~ frame 900 : 1
frame 925 ~ frame 950 : -1
frame 1210 ~ frame 1235 : -1
frame 1485 ~ frame 1510 : -1
frame 1740 ~ frame 1765 : -1
frame 2215 ~ frame 2240 : 1
frame 2240 ~ frame 2265 : 1
frame 2290 ~ frame 2315 : -1
frame 2335 ~ frame 2360 : 1
frame 2380 ~ frame 2405 : 1
frame 2455 ~ frame 2480 : -1
frame 2485 ~ frame 2510 : 1
frame 2820 ~ frame 2845 : -1
frame 2855 ~ frame 2880 : 1
frame 3105 ~ frame 3130 : -1
frame 3135 ~ frame 3160 : 1
frame 3390 ~ frame 3415 : 1
frame 4325 ~ frame 4350 : 1
frame 4455 ~ frame 4480 : 1
frame 4770 ~ frame 4795 : -1
frame 4820 ~ frame 4845 : 1
frame 4845 ~ frame 4870 : 1
frame 4900 ~ frame 4925 : 1
frame 5060 ~ frame 5085 : 1
frame 5110 ~ frame 5135 : -1
frame 5280 ~ frame 5305 : -1
frame 5430 ~ frame 5455 : -1
frame 5840 ~ frame 5865

#### 기울임이 발생하는 구간 (1에서 -1로 되는 모든 부분)까지를 1(=기울임)으로 기록하는 부분

In [ ]:
import pandas as pd
import numpy as np
import os

# Set input folder
input_folder = "D:/MultiModal/Face_z_score/A_1/"
#input_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/A_lean_time/"

#output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/A_lean_time/new_lean/"
output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/aaa_lean_time/"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Change working directory to input folder
os.chdir(input_folder)

def process_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
        if df.empty:
            return pd.DataFrame(columns=['lean time'])
        
        # Calculate box size
        df["box.size"] = df["box.height"] * df["box.width"]

        # Adjust box size
        df["box.size.adj"] = df['box.size'].rolling(5, center=False).mean()
        df = df.iloc[5::5]

        # Calculate delta
        df_delta = df['box.size.adj'].diff().iloc[1:]

        # Determine lean times based on the previous frame's box size adjusted mean
        df_lean = np.zeros(len(df_delta))
        for i in range(1, len(df_delta)):
            previous_mean = df["box.size.adj"].iloc[i-1]
            current_mean = df["box.size.adj"].iloc[i]
            threshold = previous_mean * 0.05
            delta = current_mean - previous_mean

            if abs(delta) > threshold:
                df_lean[i] = np.sign(delta)
            else:
                df_lean[i] = 0
        
        # def count_continuous_value(df, frame, d_range, d_threshold):
        #     if np.count_nonzero(df[frame:frame + d_range] == -1) >= d_threshold:
        #         df[frame:frame + d_range] = -1
        #         next_frame = frame + d_range
        #         print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : -1")
        #     elif np.count_nonzero(df[frame:frame + d_range] == 1) >= d_threshold:
        #         df[frame:frame + d_range] = 1
        #         next_frame = frame + d_range
        #         print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : 1")
        #     else:
        #         df[frame] = 0
        #         next_frame = frame + 1
        #     return next_frame 

        # frame = 0
        # while frame < len(df_lean):
        #     frame = count_continuous_value(df_lean, frame, 5, 3)

        # Mark transitions from 1 to -1 with continuous 1 and others as 0 || 1로 기록하는 것은 => 기울임이 발생했다고 보는 구간.
        df_transition = np.zeros(len(df_lean)) # 최종 배열을 저장하기 위해 0으로 모두 초기화.
        in_transition = False 
        for i in range(len(df_lean)):
            if df_lean[i] == 1: # 배열의 현재 값이 1인지 확인. 1에서 -1로 전환되는 구간이 있는지 확인. 
                in_transition = True # 있다면, true로 전환.
            if in_transition: #  True 라면, 해당 구간을 1로 전환. 
                df_transition[i] = 1
            if df_lean[i] == -1 and in_transition: # 배열의 현재 값이 -1이고, True인 경우, 그 구간이 끝난 것. 
                in_transition = False # 따라서 더 1로 기록 X
            if not in_transition and df_lean[i] != 1: # 만약, True가 아니고 , 1이 아닌 경우, 전환 구간이 또 있다는 것이므로. 
                df_transition[i] = 0 # 0으로 초기화.
        
        df_lean2_df = pd.DataFrame(df_transition, columns=['lean time'])

        #new_df = pd.concat([df, df_lean2_df], axis=1)
        
        # df_lean2_df
        total_seconds = np.arange(len(df_lean2_df)) * 0.2
        minutes, seconds = divmod(total_seconds, 60)
        hours, minutes = divmod(minutes, 60)

        time_format = [f'{int(h):02}:{int(m):02}:{s:04.1f}' for h, m, s in zip(hours, minutes, seconds)]
        df_lean2_df.index = time_format

        return df_lean2_df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame(columns=['lean time'])

for week in ["1W","2W","3W","4W"]:
    for group in ["G1","G2","G3","G4"]:
        for step in ["S1", "S2", "S3"]:
            file_name = f"Face_{week}_{group}_{step}.xlsx"
            file_path = os.path.join(input_folder, file_name)
            output_file_name = f"Face_{week}_{group}_{step}_lean.xlsx"
            output_path = os.path.join(output_folder, output_file_name)

            if os.path.exists(file_path):
                df = process_file(file_path)
                if df.empty:
                    df = pd.DataFrame(columns=['lean time'])
            else:
                df = pd.DataFrame(columns=['lean time'])

            df.to_excel(output_path, index=True)

### 이전 프레임의 얼굴 크기의 5% 만큼 차이가 나면, 기록하는 코드 (=lean_time)

In [ ]:

import pandas as pd
import numpy as np
import os

# Set input folder
input_folder = "D:/MultiModal/Face_z_score/A_1/"

#input_folder = r"C:\\Users\\user\\Desktop\\"

# 새롭게 다시 뽑아본 lean 개수. 
output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/A_lean_time/new_lean/"

#output_folder = "C:\\Users\\user\\Desktop\\result\\"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Change working directory to input folder
os.chdir(input_folder)

def process_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
        if df.empty:
            return pd.DataFrame({"lean time": []})
        
        # Calculate box size
        df["box.size"] = df["box.height"] * df["box.width"]

        # 5개의 열마다의 평균 값을 계산하고 있는 것.
        df["box.size.adj"] = df['box.size'].rolling(5, center=False).mean()
        df = df.iloc[5::5]

        # threshold의 경우, 
        #threshold = df["box.size.adj"].mean() * 0.05

        # Calculate delta
        df_delta = df['box.size.adj'].diff().iloc[1:]

        # Determine lean times
        #df_lean = np.where(df_delta.abs() > threshold, df_delta / df_delta.abs(), 0)
        
        df_lean = np.zeros(len(df_delta))
        for i in range(1, len(df_delta)):
            previous_mean = df["box.size.adj"].iloc[i-1]
            #print("이전 프레임의 얼굴 크기는 : ", previous_mean)
            current_mean = df["box.size.adj"].iloc[i]
            #print("다음 프레임의 얼굴 크기는 : ", current_mean)
            threshold = previous_mean * 0.03
            delta = current_mean - previous_mean

            if abs(delta) > threshold:
                df_lean[i] = np.sign(delta)
            else:
                df_lean[i] = 0
        
        
        def count_continuous_value(df, frame, d_range, d_threshold):
            if np.count_nonzero(df[frame:frame + d_range] == -1) >= d_threshold:
                df[frame:frame + d_range] = -1
                next_frame = frame + d_range
                print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : -1")
            elif np.count_nonzero(df[frame:frame + d_range] == 1) >= d_threshold:
                df[frame:frame + d_range] = 1
                next_frame = frame + d_range
                print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : 1")
            else:
                df[frame] = 0
                next_frame = frame + 1
            return next_frame 
        
        # 첫 프레임부터 쭉 보고자, 0 으로 설정 
        frame = 0
        while frame < len(df_lean):
            frame = count_continuous_value(df_lean, frame, 5, 3)

        df_lean2_df = pd.DataFrame(df_lean, columns=['lean time'])
        
        total_seconds = np.arange(len(new_df)) * 0.2 ## 0.2초의 경우 5프레임. || 25 프레임의 경우 0.04
        minutes, seconds = divmod(total_seconds, 60)
        hours, minutes = divmod(minutes, 60)

        time_format = [f'{int(h):02}:{int(m):02}:{s:04.1f}' for h, m, s in zip(hours, minutes, seconds)]
        df_lean2_df.index = time_format
        
        return df_lean2_df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame({"lean time": []})



# file_name = f"Front_Test.xlsx"
# file_path = os.path.join(input_folder, file_name)
# output_file_name = f"Front_Test_lean.xlsx"
# output_path = os.path.join(output_folder, output_file_name)

# if os.path.exists(file_path):
#     df = process_file(file_path)
#     if df.empty:
#         df = pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})
# else:
#     df = pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})

# df.to_excel(output_path, index=True)


# 여러 개의 파일을 돌릴 때, 아래와 같이 코드 실행. 
for week in ["1W","2W","3W","4W"]:
    for group in ["A1","A2","A3","A4"]:
        for step in ["S1", "S2", "S3"]:
            file_name = f"Face_{week}_{group}_{step}.xlsx"
            file_path = os.path.join(input_folder, file_name)
            output_file_name = f"Face_{week}_{group}_{step}_lean.xlsx"
            output_path = os.path.join(output_folder, output_file_name)

            if os.path.exists(file_path):
                df = process_file(file_path)
                if df.empty:
                    df = pd.DataFrame({"lean time": []})
            else:
                df = pd.DataFrame({"lean time": []})

            df.to_excel(output_path, index=True)

frame 15 ~ frame 40 : 1
frame 210 ~ frame 235 : -1
frame 365 ~ frame 390 : 1
frame 455 ~ frame 480 : -1
frame 615 ~ frame 640 : -1
frame 700 ~ frame 725 : 1
frame 725 ~ frame 750 : -1
frame 910 ~ frame 935 : 1
frame 940 ~ frame 965 : -1
frame 1300 ~ frame 1325 : 1
frame 1330 ~ frame 1355 : 1
frame 1500 ~ frame 1525 : 1
frame 1690 ~ frame 1715 : 1
frame 1740 ~ frame 1765 : -1
frame 1790 ~ frame 1815 : -1
frame 2050 ~ frame 2075 : 1
frame 2150 ~ frame 2175 : 1
frame 2375 ~ frame 2400 : -1
frame 2860 ~ frame 2885 : -1
frame 2980 ~ frame 3005 : -1
frame 3440 ~ frame 3465 : 1
frame 3720 ~ frame 3745 : -1
frame 3875 ~ frame 3900 : -1
frame 3930 ~ frame 3955 : 1
frame 3960 ~ frame 3985 : -1
frame 4110 ~ frame 4135 : -1
frame 4135 ~ frame 4160 : 1
frame 4415 ~ frame 4440 : 1
frame 4505 ~ frame 4530 : -1
frame 5300 ~ frame 5325 : 1
frame 5355 ~ frame 5380 : -1
frame 5800 ~ frame 5825 : 1
frame 7340 ~ frame 7365 : 1
frame 7365 ~ frame 7390 : 1
frame 7390 ~ frame 7415 : 1
frame 7430 ~ frame 7455 

# 이전 얼굴 크기보다 크면 1, 아니면 -1 그 외 모두 0

In [149]:
# 얼굴 크기 변화량 값 계산하는 구간. 
def plus_minus(df):
    df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )

    W = "box.width"
    H = "box.height"

    df["box.size.adj"] = df['box size'].rolling(5, center=False).mean()
    
    #df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
    #df_new = df.iloc[1:, :]["box delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 

    # 5 frame 형태로 뽑아오고 있는 것. 
    df = df.iloc[5::5]

    # Calculate delta
    df_delta = df['box.size.adj'].diff().iloc[1:]
    #print(df_delta)
    df_lean = np.zeros(len(df_delta))

    # threshold = 600
    for i in range(1, len(df_delta)):
        if(df_delta.iloc[i] > 600):  # 평균적인 사이즈 값
            df_lean[i] = 1 # 앞으로 기울었을 때 
        elif(df_delta.iloc[i] < -600):
            df_lean[i] = -1 # 뒤로 기울었을 때 
        else:
            df_lean[i] = 0 # 아무것도 아닐 때
    return df_lean 

In [156]:
import pandas as pd
import numpy as np
import os

# Set input folder
input_folder = "D:/MultiModal/Face_z_score/G_1/"
#input_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/A_lean_time/"

#output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/A_lean_time/new_lean/"
# 25 frame
output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Change working directory to input folder
os.chdir(input_folder)

def process_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
        if df.empty:
            return pd.DataFrame(columns=['lean time'])
        
        # Calculate box size
        df["box.size"] = df["box.height"] * df["box.width"]

        # 5개ㅛㅣㅅ
        df["box.size.adj"] = df['box.size'].rolling(5, center=False).mean()
        
        # 5 frame 형태로 뽑아오고 있는 것. 
        df = df.iloc[5::5]

        # Calculate delta
        df_delta = df['box.size'].diff().iloc[1:]

        # Determine lean times based on the previous frame's box size adjusted mean
        df_lean = np.zeros(len(df_delta))
        for i in range(1, len(df_delta)):
            previous_mean = df["box.size.adj"].iloc[i-1]
            current_mean = df["box.size.adj"].iloc[i]
            threshold = previous_mean * 0.05 # 0.01
            delta = current_mean - previous_mean

            if delta > threshold:
                df_lean[i] = 1
            elif delta < -threshold:
                df_lean[i] = -1
            else:
                df_lean[i] = 0
        
        # Modifying Code
        #new = plus_minus(df)
        
        # df_lean
        df_lean2_df = pd.DataFrame(df_lean, columns=['lean time'])

        # 25 frame
        total_seconds = np.arange(len(df_lean2_df)) * 0.04
        minutes, seconds = divmod(total_seconds, 60)
        hours, minutes = divmod(minutes, 60)

        time_format = [f'{int(h):02}:{int(m):02}:{s:04.2f}' for h, m, s in zip(hours, minutes, seconds)]
        df_lean2_df.index = time_format

        return df_lean2_df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame(columns=['lean time'])

for week in ["1W","2W","3W","4W"]:
    for group in ["G1","G2","G3","G4"]:
        for step in ["S1", "S2", "S3"]:
            file_name = f"Face_{week}_{group}_{step}.xlsx"
            file_path = os.path.join(input_folder, file_name)
            output_file_name = f"Face_{week}_{group}_{step}_lean.xlsx"
            output_path = os.path.join(output_folder, output_file_name)

            if os.path.exists(file_path):
                df = process_file(file_path)
                if df.empty:
                    df = pd.DataFrame(columns=['lean time'])
            else:
                df = pd.DataFrame(columns=['lean time'])

            df.to_excel(output_path, index=True)


### 0.2(시간 정보) 입력하기

In [ ]:
import pandas as pd
import numpy as np
import os

input_folder = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\result_C_lean\\"

out_path = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\result_A_lean\\result\\"
if not os.path.exists(out_path):
    os.makedirs(out_path)
#path = r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\C\\"

for week in ["1W", "2W", "3W", "4W"]:
    for group in ["C"]:
        for step in ["S1", "S2", "S3"]:
            
            os.chdir(input_folder)
            # box_delta_1W_A_S1
            df = pd.read_excel(f"box_delta_{week}_{group}_{step}.xlsx",index_col=0)

            seconds = df.index / 25

            minutes, seconds = divmod(seconds, 60) # divmod() 몫과 나머지를 함께 구하는 함수.
            #print(minutes)
            hours, minutes = divmod(minutes, 60)
            #print(minutes)

            # 시간 형태로 변환하여 'hh:mm:ss' 형태로 표현
            time_format = [f'{int(hours):02}:{int(minutes):02}:{seconds:.2f}' for hours, minutes, seconds in zip(hours, minutes, seconds)]

            #print(time_format)

            df.index = time_format
            
            os.chdir(out_path)
            df.to_excel(f"box_delta_{week}_{group}_{step}.xlsx")

#### 아래는 어느 구간에서 lean이 발생했는지를 확인할 수 있는 구간을 확인하는 코드

In [ ]:
df_lean2 = df_lean.copy()

def count_continuous_value(df,frame,d_range, d_threshold):
    if np.count_nonzero(df[frame:frame+d_range] ==-1) >= d_threshold:
        df[frame:frame+d_range] = -1
        next_frame = frame+d_range
        print(f"frame {(frame+1)*5} ~ frame {(frame+1)*5+d_range*5} : -1")
    elif np.count_nonzero(df[frame:frame+d_range] ==1) >= d_threshold:
        df[frame:frame+d_range] = 1
        next_frame = frame+d_range
        print(f"frame {(frame+1)*5} ~ frame {(frame+1)*5+d_range*5} : 1")
    else:
        df[frame] = 0
        next_frame = frame+1
    return next_frame

frame = 0

while (frame<len(df_lean2)):
    frame = count_continuous_value(df_lean2,frame,5,3)

In [ ]:
# # new_index 값을 초 단위로 변환하여 시간 형태로 표현
# seconds = read_file.index / 25

# minutes, seconds = divmod(seconds, 60) # divmod() 몫과 나머지를 함께 구하는 함수.
# #print(minutes)
# hours, minutes = divmod(minutes, 60)
# #print(minutes)

# # 시간 형태로 변환하여 'hh:mm:ss' 형태로 표현
# time_format = [f'{int(hours):02}:{int(minutes):02}:{seconds:.2f}' for hours, minutes, seconds in zip(hours, minutes, seconds)]

# #print(time_format)

# read_file.index = time_format

df_lean2_df = pd.DataFrame(df_lean2, columns=['lean time'])

output_path = r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\new\\Face_lean2_results.xlsx"
df_lean2_df.to_excel(output_path, index=True)

### 절대값 기준으로 box delta 값을 뽑아보는 구간.


In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

# 5분 단위로 자른 것이 아닐 때의 값을 저장하기 위한 부분. 
# output_folder = f"D:/MultiModal/Face_z_score/Face_lean_count/backup/box_delta_not_5m/"
# "D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/"
output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_a_lean_time/per_5/output/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


# r"D:\\MultiModal\\Face_z_score\\B_600\\"
# D:\MultiModal\Face_z_score\Face_lean_count\backup\All_face_box_zip\box_delta_not_5min

for group in ["A"]: # "A","B","C", "D", "E", "F", "G"
    os.chdir("D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_a_lean_time/per_5/") 
    
    # D:/MultiModal/MultiModal_Model/results/face/face_landmark/aaa_lean_time/
    
    #f"D:/MultiModal/Face_z_score/B_1/"
    
    # 5분 단위로 잘라 놓은 부분들의 diff() 값들을 구해 놓은 부분. 
    #os.chdir(f"D:/MultiModal/Face_z_score/face_landmark_Split/{group}/") 
    step_list = ["S1", "S2", "S3"]
    #step_list = ["S1_1", "S1_2", "S1_3","S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

    # 3명이 기울었을 때, 2명이 기울었을 때, 1명이 기울었을 때.
    #face_index = ["Group", "Week",] + [f"{step}_Face_lean_3p" for step in step_list] + [f"{step}_Face_lean_2p" for step in step_list] + [f"{step}_Face_lean_1p" for step in step_list]
    face_index = [f"{step}_Face_lean_3p" for step in step_list] + [f"{step}_Face_lean_2p" for step in step_list] + [f"{step}_Face_lean_1p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        # step 3개.
        for step in step_list:
            df1 = pd.read_excel(f'Face_{week}_{group}1_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'Face_{week}_{group}2_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'Face_{week}_{group}3_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'Face_{week}_{group}4_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}4_{step}.xlsx') else []
            
            
            #print(df1)
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
        
            length = a.min()-1 # 최솟값에서 -1을 해줌. 
            
            if len(df1) > 0:
                df1 = plus_minus(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"box delta_B1": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"box delta_B2": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"box delta_B3": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"box delta_B4": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
            
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = 1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            
            df_concat.to_excel(output_folder + f"box_delta_{week}_{group}_{step}.xlsx")

C:\Users\user\AppData\Local\Temp\ipykernel_5752\2367006766.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 0 # 아무것도 아닐 때
C:\Users\user\AppData\Local\Temp\ipykernel_5752\2367006766.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 1 # 앞으로 기울었을 때
C:\Users\user\AppData\Local\Temp\ipykernel_5752\2367006766.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = -1 # 뒤로 기울었을 때
C:\Users

## box_delta 1~4 중 각 조건에 맞게 12프레임 중 8개가 -1 이라면 -1로 쭉 기록 || 1이라면 1로 쭉 기록

## -1 혹은 1이 10개 이상 되는 행을 찾아보자

In [ ]:
def find_consecutive_values(series, value, min_count):
    count = 0
    start_index = -1
    ranges = []
    
    for i, v in enumerate(series):
        if v == value:
            if count == 0:
                start_index = i
            count += 1
        else:
            if count >= min_count:
                ranges.append((start_index, i - 1))
            count = 0
    
    if count >= min_count:
        ranges.append((start_index, len(series) - 1))
    
    return ranges

### 첫 행부터 50행(=25 프레임)까지, 1 혹은 -1의 값이 얼마나 있을까

In [ ]:
def sliding_window_count(df, window_size, value_list, min_count):
    n = len(df)
    for start in range(n - window_size + 1):
        end = start + window_size
        subset_df = df.iloc[start:end]
        count = subset_df['box delta.2'].isin(value_list).sum()
        if count >= min_count:
            return start, end - 1, count
    return None

## 1이 0~50행 내에서 20개 이상 있는지 확인하기 위한 것

In [ ]:
def sliding_window_count_ones(df, window_size, target_value, min_count): # window_size => 50개의 행만큼 건너뛰어서 보고 싶은 것. 
    # target_value = 1(or 1) # min_count = 20개(* 80%)
    n = len(df)
    list1 = []
    for start in range(0, n, window_size): # window_size 만큼 건너뛰면서 확인. 
        end = start + window_size
        subset_df = df.iloc[start:end]
        count = (subset_df['box delta.1'] == target_value).sum() # lean_3p
        # count >=12 이라면, 갯수가 몇 개인 지를 카운트 해주는 부분
        if count >= min_count:
            list1.append((start, end - 1, count))     
            
    return list1

## Box delta ~ box delta 3까지 값 전체 아래에 해당 값(1 or -1)이 있는지 확인하고, 몇 개 있는지 셀 수 있는 코드

In [ ]:
def sliding_window_count_ones(df, window_size, target_value, min_count):
    n = len(df)
    columns_to_check = ['box delta', 'box delta.1', 'box delta.2', 'box delta.3']
    
    for start in range(0, n, window_size):
        end = min(start + window_size, n)  # Ensure we do not go out of bounds
        subset_df = df.iloc[start:end]
        
        # Check if any of the columns in the current window meet the condition
        for column in columns_to_check:
            count = (subset_df[column] == target_value).sum()
            if count >= min_count:
                df.iloc[start:end, df.columns.get_indexer(columns_to_check)] = target_value
                break  # Once one column meets the condition, we update the entire window and move on
                
    return df

In [ ]:
import os 
import pandas as pd

## box.delta 값이 연속으로 1인 값이 7개 이상일 때의 열을 찾는 코드

In [ ]:
path = r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/05_27_box_delta_zip/C_250/"
# D:\MultiModal\Face_z_score\Face_lean_count\backup\All_face_box_zip\05_27_box_delta_zip\C_250
#print(os.getcwd())

os.chdir(path)
#print(os.getcwd())
for week in ["1W", "2W", "3W", "4W"]:
    for step in ["S1", "S2", "S3"]:
        read_data = pd.read_excel(f"box_delta_diff_{week}_C_{step}.xlsx", index_col=0)
        
        # -1 값이 7개 이상으로 나오는 곳 
        consecutive_ranges = find_consecutive_values(read_data['box delta.1'], 1, 8)
        for start, end in consecutive_ranges:
            print(f"연속된 1 값이 {week}_C_{step}파일에서, {start}에서 {end}까지 발견됨")

## 0~N행까지 계속 탐색해보았을 때, 1 혹은 -1이 연속으로 N개 이상 있는 곳을 찾는 코드

In [ ]:
def sliding_window_count_values(df, window_size, target_value_1, target_value_2, min_count):
    n = len(df) # 30000
    columns_to_check = ['box delta.3'] # 'box delta.1', 'box delta.2','box delta.3',
    
    for start in range(n - window_size + 1):
        end = start + window_size
        subset_df = df.iloc[start:end]
        
        # Check counts for both target values
        count_1 = (subset_df[columns_to_check] == target_value_1).sum().sum()
        count_2 = (subset_df[columns_to_check] == target_value_2).sum().sum()
        
        if count_1 >= min_count or count_2 >= min_count:
            if count_1 > count_2:
                df.iloc[start, df.columns.get_indexer(columns_to_check)] = target_value_1
            elif count_2 > count_1:
                df.iloc[start, df.columns.get_indexer(columns_to_check)] = target_value_2
        
        # if count_1 >= min_count:
        #     df.iloc[start, df.columns.get_indexer(columns_to_check)] = target_value_1
        # elif count_minus_1 >= min_count:
        #     df.iloc[start, df.columns.get_indexer(columns_to_check)] = target_value_2
                
    return df

## 만족하는 열이 총 몇 개가 있는지를 확인해보는 구간

In [ ]:
path = r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/05_27_box_delta_zip/C_250/"
# D:\MultiModal\Face_z_score\Face_lean_count\backup\All_face_box_zip\05_27_box_delta_zip\C_250
#input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check\\"
#print(os.getcwd())

os.chdir(path)
#print(os.getcwd())
for week in ["1W", "2W", "3W", "4W"]:
    for step in ["S1", "S2", "S3"]:
        read_data = pd.read_excel(f"box_delta_diff_{week}_C_{step}.xlsx", index_col=0)
        
        # 첫 50행씩 이동하며 1 또는 -1의 개수를 확인
        #result = sliding_window_count(read_data, 50, [-1, 1], 20)
        result = sliding_window_count_ones(read_data, 12, -1, 8)    
        # 결과 출력
        if result:
            for start, end, count in result:
                print(f"{week}_C_{step}파일에서, 행 {start}에서 {end}까지의 구간에서 값이 -1인 개수는 {count}입니다.")
        else:
            print("조건을 만족하는 구간이 없습니다.")

## 만족하는 열이 있다면, 그 부분을 모두 -1 혹은 1로 변경해주는 코드 부분

In [ ]:
# 예제 데이터 프레임

data = {
    'box delta': [0, 0, 1, 1, -1, -1, 0, 0, 1, 1, -1, -1, 0, 0]
}

df = pd.DataFrame(data)

# 함수 호출
window_size = 5
target_value_1 = 1
target_value_2 = -1
min_count = 2

result_df = sliding_window_count_values(df, window_size, target_value_1, target_value_2, min_count)
print(result_df)

In [ ]:
import os
import pandas as pd
path = r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/05_27_box_delta_zip/C_400/"
if not os.path.exists(path):os.makedirs(path)
# input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check\\"
#print(os.getcwd())

os.chdir(path)
#print(os.getcwd())
for week in ["1W", "2W", "3W", "4W"]:
    for step in ["S1", "S2", "S3"]:
        read_data = pd.read_excel(f"box_delta_diff_{week}_C_{step}.xlsx", index_col=0)
        # box_delta_diff_1W_C_S1
        
        # 첫번째부터 12행씩 이동하며 1 또는 -1의 개수를 확인
        result = sliding_window_count_values(read_data, 12, 1, -1, 8)    
        # 결과 출력
        
        # print(result)
        
        result_path = f"modified_box_delta_{week}_C_{step}.xlsx"
        result.to_excel(result_path)
        print(f"{result_path} 파일이 저장되었습니다.")

### Lean time을 한곳에 저장할 수 있는 코드

In [165]:
import os
import pandas as pd 
import numpy as np

data = []

# "D:/MultiModal/MultiModal_Model/results/face/face_landmark/cc_lean_time/output/"

# 5개 중에 3개 평균 5% 변화
# r"D:\\MultiModal\\Face_z_score\\C_Lean_Time\\"

# r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/ggg_lean_time/output/"

Out_folder = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/"
if not os.path.exists(Out_folder):
    os.makedirs(Out_folder)

path = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/"
# "D:/MultiModal/MultiModal_Model/results/face/face_landmark/G_lean_time/new_lean/"
#  r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/gg_lean_time/"

for group in ["G"]: # "C"
    for week in ["1W","2W","3W","4W"]: # "2W","3W","4W"
        for step in ["S1", "S2", "S3"]: # , "S2", "S3"
            # "S1_1", "S1_2", "S1_3","S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"
            os.chdir(path)
            
            # 각 파일에서 lean_time 열을 추출
            lean_times = []
            for i in range(1, 5):
                file_path = f"Face_{week}_{group}{i}_{step}_lean.xlsx"
                if os.path.exists(file_path):
                    read_file = pd.read_excel(file_path)
                    lean_times.append(read_file[['lean time']])
                else:
                    # 파일이 존재하지 않는 경우 빈 데이터 프레임 추가
                    lean_times.append(pd.DataFrame({"lean time": []}))
                    print(f"{file_path} 파일이 존재하지 않습니다. 빈 데이터 프레임이 사용됩니다.")
                
                
                #read_file = pd.read_excel(file_path)
                #lean_times.append(read_file[['lean time']])

             # 모든 lean time 열을 하나의 DataFrame으로 결합 (가로로)
            combined_lean_times = pd.concat(lean_times, axis=1, ignore_index=True)
            
            # 열 이름 설정
            combined_lean_times.columns = [f'lean_time_{group}{i}' for i in range(1, 5)]
            
            # 엑셀 파일로 저장
            combined_output_path = os.path.join(Out_folder, f"Combined_lean_time_{week}_{group}_{step}.xlsx")
            combined_lean_times.to_excel(combined_output_path, index=True)
            print(f"Combined_lean_time_{week}_{group}_{step} 파일이 기록되었습니다.")

Combined_lean_time_1W_G_S1 파일이 기록되었습니다.
Combined_lean_time_1W_G_S2 파일이 기록되었습니다.
Combined_lean_time_1W_G_S3 파일이 기록되었습니다.
Combined_lean_time_2W_G_S1 파일이 기록되었습니다.
Combined_lean_time_2W_G_S2 파일이 기록되었습니다.
Combined_lean_time_2W_G_S3 파일이 기록되었습니다.
Combined_lean_time_3W_G_S1 파일이 기록되었습니다.
Combined_lean_time_3W_G_S2 파일이 기록되었습니다.
Combined_lean_time_3W_G_S3 파일이 기록되었습니다.
Combined_lean_time_4W_G_S1 파일이 기록되었습니다.
Combined_lean_time_4W_G_S2 파일이 기록되었습니다.
Combined_lean_time_4W_G_S3 파일이 기록되었습니다.


## Box delta 간의 diff() 값을 구했으니, 이에 맞게 몇 명이 기울었는지를 체크하는 구간.

In [172]:
import os
import pandas as pd 
import numpy as np

data = []

### 5분 단위로 자르지 않은 부분들을 모아놓은 곳. 
#out_folder = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\Output_C\\"

# 5분 단위로 나누지 않고, 그대로 둔 것들을 기록해둔 파일.
#input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\box_delta_not_5min\\"

#Out_folder =  r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/gg_lean_time/output/lean_3p/"

# 새롭게 뽑아본 5개중 3개, 이전 프레임의 5% 변화
#Out_folder = r"D:\\MultiModal\\Face_z_score\\G_Lean_Time\\output_3p\\"
# 절대값 기준으로 뽑아보는 파일 경로 
#Out_folder =  r"D:\\MultiModal\\Face_z_score\\B_600\\output_3p\\"
Out_folder = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/lean_3p/"

# r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/ggg_lean_time/output/lean_3p/"
if not os.path.exists(Out_folder):
    os.makedirs(Out_folder)

#input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\C\\"

#path = r"D:\\MultiModal\\Face_z_score\\G_Lean_Time\\

# 절대값 기준
#path = r"D:\\MultiModal\\Face_z_score\\B_600\\"
path = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/"

for group in ["G"]:
    for week in ["1W","2W","3W","4W"]:
        for step in ["S1", "S2", "S3"]: # , "S2", "S3"
            # "S1_1", "S1_2", "S1_3","S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"
            os.chdir(path)
            read_file = pd.read_excel(f"Combined_lean_time_{week}_{group}_{step}.xlsx")
            # Combined_lean_time_1W_C_S1
            # box_delta_
            
            step_data_3p = []
            step_data_2p = []
            step_data_1p = []
            
            # 동시에 기울어진 사람들을 카운트 하는 구간. 
            read_file['same']=0 
                    
            for i in range (len(read_file)):
                #print(read_file.iloc[i, 1:5])
                val_count = read_file.iloc[i,1:5].value_counts()
                # val_count 안에 있는 값이 0 이라면 0으로 초기화. 
                val_count[0] = 0
                # 최댓값을 넣어주려는 부분. 
                #print(read_file.iloc[i, 5:])
                read_file.iloc[i,5] = val_count.max()
                
            same = np.array(read_file["same"])

            # 동시에 4명 중 N명이 기울었을 때를 세는 부분 || 모두 list 안에 들어있는 것. 
            step_data_3p = np.where(same>=3,1,0)
            step_data_2p = np.where(same>=2,1,0)
            step_data_1p = np.where(same>=1,1,0)
        
            # lean_4p, 3p,2p,1p에 맞게 데이터 값을 데이터 프레임으로 지정. 
            lean_3p = pd.DataFrame({"lean_3p" : step_data_3p})
            lean_2p = pd.DataFrame({"lean_2p" : step_data_2p})
            lean_1p = pd.DataFrame({"lean_1p" : step_data_1p})    
            
            # 데이터 프레임 합치는 구간. 
            new_df = pd.concat([read_file, lean_3p], axis=1)
            new_df_1 = pd.concat([new_df, lean_2p], axis=1)
            new_df_2 = pd.concat([new_df_1, lean_1p], axis=1)
            
            # Unnamed 0 열 지우기.
            #new_df_2.drop(['Unnamed 0'], axis= 1, inplace=True)
            
            # 엑셀 파일 값으로 저장하는 부분.
            os.chdir(Out_folder) 
            new_df_2.to_excel(f"Box_delta_{week}_{group}_{step}.xlsx")
            print(f"Box_delta_{week}_{group}_{step} 파일이 기록되었습니다.")

Box_delta_1W_G_S1 파일이 기록되었습니다.
Box_delta_1W_G_S2 파일이 기록되었습니다.
Box_delta_1W_G_S3 파일이 기록되었습니다.
Box_delta_2W_G_S1 파일이 기록되었습니다.
Box_delta_2W_G_S2 파일이 기록되었습니다.
Box_delta_2W_G_S3 파일이 기록되었습니다.
Box_delta_3W_G_S1 파일이 기록되었습니다.
Box_delta_3W_G_S2 파일이 기록되었습니다.
Box_delta_3W_G_S3 파일이 기록되었습니다.
Box_delta_4W_G_S1 파일이 기록되었습니다.
Box_delta_4W_G_S2 파일이 기록되었습니다.
Box_delta_4W_G_S3 파일이 기록되었습니다.


## Same열이 있다면, 보이지 않도록 지우기

In [179]:
import os
import pandas as pd

################### 현재 코드 : Add_Lean_3p_A : 'A'그룹의 same열(=불필요) 지우고, box delta 이름 알아보기 쉽게 변경하고 저장한 경로. 
#out_path = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Add_Lean_3p_A\\"


### r"D:\\MultiModal\\Face_z_score\\C_600\\output_3p\\"
################### 현재 코드 : modify2_C : 'A'그룹(=lean 400 기준) same열(=불필요) 지우고, box delta 이름 알아보기 쉽게 변경하고 저장한 경로. 
### out_path = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\modified_G_lean_time\\"

#out_path =  r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/gg_lean_time/output/lean_3p/rename/"
# r"D:\\MultiModal\\Face_z_score\\G_Lean_Time\\output_3p\\"

# 절대값 기준 
#out_path = r"D:\\MultiModal\\Face_z_score\\B_600\\output_3p\\backup\\"
# r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/output/lean_3p/"
out_path = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/lean_3p/"
if not os.path.exists(out_path):
    os.makedirs(out_path)
# r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/ggg_lean_time/output/lean_3p/rename/"


################### 현재 코드 : C : 'A'그룹(=lean 400 기준) same열(=불필요) 지우고, box delta 이름 알아보기 쉽게 변경하고 저장한 경로.  
# r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\C_400_output\\"    

for group in ["G"]:
    for week in ["1W","2W","3W","4W"]:
        # 5분 단위로 나누어서 필요없는 데이터 지우기. 
        for step in ["S1", "S2", "S3"]:

            os.chdir(r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/lean_3p/")
            # r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/aaa_lean_time/output/lean_3p/"
            
            # r"D:\\MultiModal\\Face_z_score\\B_600\\output_3p\\"
            
            #### 경로 설명 : C_250의 경우, C 그룹에서 (box delta 값 수정 X) 값에서 lean 3p, 2p, 1p만 추가한 곳.
            #r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\C_250\\"
            
            #### 경로 설명 : Check_(A/C)250의 경우, (A/C)그룹의 box_delta 값을 모두 1 혹은 -1로 통일한 값 기록
            #r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check_C250\\"
            
            #### 경로 설명 : A 400의 경우, (A/C)그룹을 400 기준으로 뽑아본 것 
            #r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\A_400\\"
            
            read_file = pd.read_excel(f"Box_delta_{week}_{group}_{step}.xlsx", index_col=0)

            # Unnamed: 0 이라는 이름을 가진 column 지우기. 
            if 'Unnamed: 0' in read_file.columns:
                read_file.drop(['Unnamed: 0'], axis=1, inplace=True)

            # same 열 지우기.
            if 'same' in read_file.columns:
                read_file.drop(['same'], axis=1, inplace=True)
                
            # read_file.rename(columns = {
            #     "box delta_A1": "box_delta_A1",
            #     "box delta_A2": "box_delta_A2",
            #     "box delta_A3": "box_delta_A3",
            #     "box delta_A4": "box_delta_A4"}, inplace=True)    
                            
            read_file.to_excel(out_path + f"Box_delta_{week}_{group}_{step}.xlsx")

#### 20분을 5분씩 4개의 파일로 끊어서 저장해보자

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
%matplotlib inline

In [ ]:
import re
import math

for group in ["A","B","C","D","E","F","G"]: # D:\MultiModal\Face_z_score
    os.chdir(f"D:\\MultiModal\\Face_z_score\\{group}\\")
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())
    
    # 출력할 파일 불러오기
    output_folder = f"D:\\MultiModal\\Face_z_score\\face_landmark_Split\\{group}\\"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in file_list:
        if file.split('.')[-1] == "xlsx": 
            data_df = pd.read_excel(file)
            data_index = data_df.iloc[:, 1:].columns.to_list()

            file_name = file.split('.')[0]
            #print(file_name.split('_')[4])

            if file_name.split('_')[4] == "S3" or file_name.split('_')[4] == "Baseline":
                data_df.iloc[:, 1:].to_excel(output_folder + file_name + '.xlsx')
            
            else:
                
                num_splits = 4
                
                rows_per_split = len(data_df) // num_splits

                for i in range(num_splits):
                    # 마지막 부분 처리
                    if i == num_splits - 1:
                        split_df = data_df[i * rows_per_split:]
                    else:
                        split_df = data_df[i * rows_per_split:(i + 1) * rows_per_split]
                        
                    split_df.iloc[:, 1:].to_excel(output_folder + f'{file_name}_{i + 1}.xlsx', index=data_index)

                print(f'{num_splits}개의 파일로 나누기 완료.')

## Lean_1p에 대한 값의 합을 구해보자. 

In [ ]:
import os
import pandas as pd

# 합산 값을 보기 위한 output 경로 
output = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_a_lean_time/per_5/SUM_3p/"

data = []

for group in ["A"]:
    for week in ["1W", "2W", "3W", "4W"]:
        for step in ["S1_1","S1_2","S1_3","S1_4","S2_1","S2_2","S2_3","S2_4","S3"]:
            # 현재 디렉토리로 변경 
            os.chdir(r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\All_face_lean_5m\\") 
            read_data = pd.read_excel(f"box_delta_{week}_{group}_{step}.xlsx", index_col=0)
        
            result = read_data["lean_1p"].sum()
            
            data.append(result)
            print(data)

temp_df = pd.DataFrame(data, columns=["SUM_LEAN_1P"])
temp_df.to_excel(output + "LEAN_1P_SUM.xlsx")

## Lean 4p 열을 추가해보자 (same = Lean 4p) 값이랑 똑같음

In [2]:
import os
import numpy as np
import pandas as pd 

################### 현재 코드 : Add_Lean_4p_(A/C) : '(A/C)'그룹에 lean_4p 값을 추가하고 저장한 경로. || 기준 400

# 600으로 추출해보았을 때의 최종 결과물.
#output = r"D:\\MultiModal\\Face_z_score\\B_600\\output_3p\\backup\\output_4p\\"
# r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/persent_1/output/lean_3p/rename/"
#output = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/ggg_lean_time/output/lean_3p/rename/lean_4p/"
output = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/lean_3p/Lean_4p/"

if not os.path.exists(output):
    os.makedirs(output)

# D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Modify_box_size_all\\Modify_time_5m

################### 5분으로 나누지 않은 값을 기록한 부분. 
# D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/Modify_box_size_all/

################### 현재 코드 : modify1_C : 'C'그룹(=lean 250 기준) same열(=불필요) 지우고, box delta 이름 알아보기 쉽게 변경하고 저장한 경로. 
#input_path = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\modify1_C\\"

for week in ["1W", "2W", "3W", "4W"]:
    for group in ["G"]:
        for step in ["S1", "S2", "S3"]:
            # ["S1_1", "S1_2", "S1_3", "S2_1", "S2_2", "S2_3", "S3"]:
            # ["S1", "S2", "S3"]:
            
            # 현재 디렉토리로 변경 
            #os.chdir(r"D:\\MultiModal\\Face_z_score\\B_600\\output_3p\\backup\\")
            
            os.chdir(r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/lean_3p/")
            # r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\modify2_A\\"
            
            # r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/persent_1/output/lean_3p/rename/
            # r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/ggg_lean_time/output/lean_3p/rename/")
            
            #### 경로 설명 : 5m 간격으로 자르지 않은 것들을 저장한 곳 
            #os.chdir(r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/Modify_time") 
            
            #### 경로 설명 : 5m 간격으로 자른 것들을 저장한 곳. 
            # os.chdir(r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/Modify_time/Modify_time_5m")
            
            # C 그룹에서 box delta의 값을 바꿔주는 부분을 저장한 곳. 
            # os.chdir(r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\Rename_C\\")
            
            #파일을 읽어옴. 
            read_data = pd.read_excel(f"Box_delta_{week}_{group}_{step}.xlsx", index_col=0)
           
            # lean_4p를 저장하기 위한 리스트 선언. 
            step_data_4p = []
        
            # 동시에 기울어진 사람들을 카운트 하는 구간. 
            read_data['same'] = 0 
                    
            for i in range (len(read_data)):
                # box_size 1~4 까지 있는 것들을 세어야 하는 것이므로. 
                #print(read_data.iloc[i, :4])
                val_count = read_data.iloc[i, :4].value_counts()
                # val_count 안에 있는 값이 0 이라면 0으로 초기화
                val_count[0] = 0
                
                #print(read_data.iloc[i, :8])
                #print(read_file.iloc[i, 5])
                
                # 최댓값을 넣어주려는 부분. 
                #print(read_data.iloc[i, 6:])
                read_data.iloc[i, 7] = val_count.max()
            
            same = np.array(read_data["same"])

            # 동시에 4명 중 N명이 기울었을 때를 세는 부분 || 모두 list 안에 들어있는 것. 
            step_data_4p = np.where(same>=4,1,0)
            
            # lean_4p에 맞게 데이터 값을 데이터 프레임으로 지정. 
            lean_4p = pd.DataFrame({"lean_4p" : step_data_4p})
            new_df = pd.concat([read_data, lean_4p], axis=1)
    
            if 'same' in new_df.columns:
                new_df.drop(['same'], axis=1, inplace=True) 
            
            # 위치 바꿔주는 부분 넣어줄 것. 
            new_df.insert(4, 'lean_4p', new_df.pop('lean_4p'))    
            # print(new_df)
            #엑셀 파일 값으로 저장하는 부분. 
            new_df.to_excel(output + f"box_delta_{week}_{group}_{step}.xlsx")       


## Lean 4p의 합계를 계산해보자

In [186]:
import os

# Lean_4P 몇 개인지 볼 것. 
###output = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\backup\All_face_box_zip\\05_27_box_delta_zip\\Lean_time_sum_A\\"

# lean_3p 값 몇 개인지 출력해보기. => 600 
#output = r"D:\\MultiModal\\Face_z_score\\B_600\\Lean_3p_SUM\\"
#output = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/aa_lean_time/output/lean_3p/rename/lean_4p/3p_SUM/"

output = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/SUM_3p/"

#output = r"D:\\MultiModal\\Face_z_score\\G_Lean_Time\\output_3p\\backup\\output_4p\\SUM_Lean_3p\\"
if not os.path.exists(output):
    os.makedirs(output)

data = []
for week in ["1W", "2W", "3W", "4W"]:
    for group in ["G"]:
        for step in ["S1", "S2", "S3"]:
            # "S1_1", "S1_2", "S1_3", "S2_1", "S2_2", "S2_3", 
            
            # lean_3p
            #os.chdir(r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/aa_lean_time/output/lean_3p/rename/lean_4p/") 
            os.chdir(r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/25_g_lean_time/per_5/output/lean_3p/")
            
            # lean 4p를 계산하기 위한 코드 (A그룹)
            # r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\result_A_lean\\"
            read_data = pd.read_excel(f"box_delta_{week}_{group}_{step}.xlsx", index_col=0)
        
            result = read_data["lean_3p"].sum()
            
            data.append(result)
            
            #데이터 확인해볼 것 
            # print(data)

temp_df = pd.DataFrame(data, columns=["SUM_LEAN_3P"])
os.chdir(output)
temp_df.to_excel("LEAN_3p_SUM_B.xlsx")